# RNN기반 분류기

In [1]:
# 데이터 로딩
from sklearn.datasets import fetch_20newsgroups

categories = ['comp.graphics', 'sci.space', 'rec.sport.baseball']
newsgroups = fetch_20newsgroups(subset='all', categories=categories)
X = newsgroups.data
y = newsgroups.target
print(newsgroups.target_names)
print(X[0])
print(y[0])


['comp.graphics', 'rec.sport.baseball', 'sci.space']
From: kjenks@gothamcity.jsc.nasa.gov
Subject: Life on Mars???
Organization: NASA/JSC/GM2, Space Shuttle Program Office 
X-Newsreader: TIN [version 1.1 PL8]
Lines: 12

I know it's only wishful thinking, with our current President,
but this is from last fall:

     "Is there life on Mars?  Maybe not now.  But there will be."
        -- Daniel S. Goldin, NASA Administrator, 24 August 1992

-- Ken Jenks, NASA/JSC/GM2, Space Shuttle Program Office
      kjenks@gothamcity.jsc.nasa.gov  (713) 483-4368

     "The man who makes no mistakes does not usually make
      anything."
        -- Edward John Phelps, American Diplomat/Lawyer (1825-1895)

2


In [ ]:
# 데이터전처리
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
max_len = 200

tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X)   # 토크나이저 단어 사전 학습
X_encoded = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_encoded, maxlen=max_len) # 길이를 max_len으로 통일(패딩/자르기)
print(X_padded.shape)   #(문서수,max_len)

(2954, 200)


In [ ]:
# 데이터분리/텐서변환
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# 전체데이터에서 train/valid.test로 분리
X_train, X_test, y_train, y_test = train_test_split(torch.tensor(X_padded), torch.tensor(y), test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# dataset/dataloader
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# 모델 생성
import torch.nn as nn

# 임베딩 -> LSTM -< FC로 문장을 요약해 3개 클래스 로짓을 출력하는 분류기 모델
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super().__init__()
        # embedding - lstm - dense
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)   # (B,Y,*)
        self.fc = nn.Linear(hidden_size, 3) # 마지막 은닉 -> 3개 클래스 로짓 반환

    def forward(self, x):
        x = self.embedding(x)       # (B, T) -> (B, T, E)
        _, (h, c) = self.lstm(x)    # h: (L, B, H), C: (L, B, H)
        out = self.fc(h[-1])        # 마지막 레이어 은닉상태 (B, H) -> (B, 3)
        return out

In [ ]:
# 모델 학습
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # cuda 또는 cpu
embedding_dim = 100
hidden_size = 128

model = LSTMClassifier(vocab_size, embedding_dim, hidden_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습루프
train_losses, train_accs = [], []
val_losses, val_accs = [], []

epochs = 50
for epoch in range(epochs):

    # 학습
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.detach().cpu().item()
        pred = output.argmax(dim=1) # 가장 큰 로짓 인덱스로 반환
        train_correct += (pred == y_batch).sum().detach().cpu().item()
        train_total += len(y_batch)

    train_loss /= len(train_loader)
    train_acc = train_correct / train_total
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # 검증
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            output = model(X_batch)
            loss = criterion(output, y_batch)

            val_loss += loss.detach().cpu().item()
            pred = output.argmax(dim=1)
            val_correct += (pred == y_batch).sum().detach().cpu().item()
            val_total += len(y_batch)

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total
        val_losses.append(val_loss)
        val_accs.append(val_acc)

    # 출력(train_loss, val_loss)
    print(f'Epoch {epoch + 1}/{epochs}: '
          f'Train Loss {train_loss:.4f}, '
          f'Train Acc {train_acc:.4f}, '
          f'Val Loss {val_loss:.4f}, '
          f'Val Acc {val_acc:.4f}, ')


Epoch 1/50: Train Loss 1.0565, Train Acc 0.4635, Val Loss 1.0042, Val Acc 0.5349, 
Epoch 2/50: Train Loss 0.9064, Train Acc 0.6032, Val Loss 0.8640, Val Acc 0.6195, 
Epoch 3/50: Train Loss 0.7147, Train Acc 0.7106, Val Loss 0.7429, Val Acc 0.7019, 
Epoch 4/50: Train Loss 0.5350, Train Acc 0.8011, Val Loss 0.7126, Val Acc 0.7252, 
Epoch 5/50: Train Loss 0.3922, Train Acc 0.8540, Val Loss 0.6159, Val Acc 0.7653, 
Epoch 6/50: Train Loss 0.2631, Train Acc 0.9085, Val Loss 0.5761, Val Acc 0.7780, 
Epoch 7/50: Train Loss 0.2181, Train Acc 0.9296, Val Loss 0.8745, Val Acc 0.7188, 
Epoch 8/50: Train Loss 0.2470, Train Acc 0.9196, Val Loss 0.5701, Val Acc 0.8076, 
Epoch 9/50: Train Loss 0.1068, Train Acc 0.9693, Val Loss 0.5674, Val Acc 0.8288, 
Epoch 10/50: Train Loss 0.0753, Train Acc 0.9820, Val Loss 0.6352, Val Acc 0.8330, 
Epoch 11/50: Train Loss 0.0519, Train Acc 0.9899, Val Loss 0.6052, Val Acc 0.8224, 
Epoch 12/50: Train Loss 0.0475, Train Acc 0.9926, Val Loss 0.7752, Val Acc 0.7780, 
E

In [ ]:
# 모델 평가
# - 정답, 모델 예측값을 가지고, classification_report 작성
from sklearn.metrics import classification_report

model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        output = model(X_batch)
        loss = criterion(output, y_batch)
        pred = output.argmax(dim=1)

        all_preds.extend(pred.detach().cpu().numpy())       # 배치 예측을 리스트에 누적
        all_labels.extend(y_batch.detach().cpu().numpy())   # 배치 정답을 리스트에 누적

print(classification_report(all_labels, all_preds, target_names=newsgroups.target_names))

                    precision    recall  f1-score   support

     comp.graphics       0.83      0.81      0.82       202
rec.sport.baseball       0.84      0.88      0.86       202
         sci.space       0.74      0.73      0.74       187

          accuracy                           0.81       591
         macro avg       0.81      0.81      0.81       591
      weighted avg       0.81      0.81      0.81       591



## 사전학습된 임베딩 적용하기

In [7]:
%pip install gensim -q

Note: you may need to restart the kernel to use updated packages.


- ted_en_fasttext.model : 모델 본체(메타데이터 + 학습된 파라미터 경로)
- ted_en_fasttext.model.wv.vectors_ngrams.npy : FastText의 핵심 특징이 서브워드(n-gram)벡터 행렬(numpy 배열 파일)

In [ ]:
from gensim.models import FastText

fasttext_model = FastText.load('ted_en_fasttext.model')
print(fasttext_model.vector_size)   # 단어 벡터 차원(단어 임베딩 벡터의 차원수)

100


In [ ]:
import numpy as np

embedding_dim = fasttext_model.vector_size
embedding_matrix = np.zeros((vocab_size, embedding_dim)) # (단어사전크기, 임베딩차원) 0 행렬 초기화

word_index = tokenizer.word_index # 38000
word_index = {word:index \
              for word, index in word_index.items() \
                if index < vocab_size}
print(len(word_index)) # 10000

for word, index in word_index.items():
    if word in fasttext_model.wv:   # FastText가 해당 벡터를 가지고 있으면
        embedding_matrix[index] = fasttext_model.wv[word]   # 해당 인덱스 위치에 임베딩 벡터를 채움

9999


In [15]:
# 모델 생성
import torch.nn as nn

# 사전학습 임베딩(embedding_matrix)으로 임베딩 레이어를 초기화하면 LSTM 분류기
class LSTMClassifier2(nn.Module):
    def __init__(self, vocab_size, embedding_dim, embedding_matrix, hidden_size):
        super().__init__()
        # embedding - lstm - dense
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))    # 사전학습 임베딩 가중치로 초기화
        self.embedding.weight.requires_grad = True  # 임베딩을 학습에 포함할지 여부 (True = 미세조정, False 고정)

        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 3)

    def forward(self, x):
        x = self.embedding(x)       # (B, T) -> (B, T, E)
        _, (h, c) = self.lstm(x)    # h: (L, B, H)
        out = self.fc(h[-1])        # 마지막 레이어 ㅇㄴ닉 사용 (B,H) -> (B,3)
        return out


In [13]:
# 모델 학습
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # cuda 또는 cpu
embedding_dim = 100
hidden_size = 128

model = LSTMClassifier2(vocab_size, embedding_dim, embedding_matrix, hidden_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 학습루프
train_losses, train_accs = [], []
val_losses, val_accs = [], []

epochs = 100
for epoch in range(epochs):

    # 학습
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.detach().cpu().item()
        pred = output.argmax(dim=1)
        train_correct += (pred == y_batch).sum().detach().cpu().item()
        train_total += len(y_batch)

    train_loss /= len(train_loader)
    train_acc = train_correct / train_total
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # 검증
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            output = model(X_batch)
            loss = criterion(output, y_batch)

            val_loss += loss.detach().cpu().item()
            pred = output.argmax(dim=1)
            val_correct += (pred == y_batch).sum().detach().cpu().item()
            val_total += len(y_batch)

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total
        val_losses.append(val_loss)
        val_accs.append(val_acc)

    # 출력(train_loss, val_loss)
    print(f'Epoch {epoch + 1}/{epochs}: '
          f'Train Loss {train_loss:.4f}, '
          f'Train Acc {train_acc:.4f}, '
          f'Val Loss {val_loss:.4f}, '
          f'Val Acc {val_acc:.4f}, ')


Epoch 1/100: Train Loss 1.0972, Train Acc 0.3550, Val Loss 1.0947, Val Acc 0.3890, 
Epoch 2/100: Train Loss 1.0939, Train Acc 0.3794, Val Loss 1.0917, Val Acc 0.4186, 
Epoch 3/100: Train Loss 1.0905, Train Acc 0.4249, Val Loss 1.0877, Val Acc 0.4567, 
Epoch 4/100: Train Loss 1.0866, Train Acc 0.4556, Val Loss 1.0830, Val Acc 0.4926, 
Epoch 5/100: Train Loss 1.0812, Train Acc 0.4984, Val Loss 1.0769, Val Acc 0.5095, 
Epoch 6/100: Train Loss 1.0723, Train Acc 0.4857, Val Loss 1.0627, Val Acc 0.4968, 
Epoch 7/100: Train Loss 1.0305, Train Acc 0.5153, Val Loss 0.9507, Val Acc 0.6068, 
Epoch 8/100: Train Loss 0.9351, Train Acc 0.5852, Val Loss 0.9151, Val Acc 0.5962, 
Epoch 9/100: Train Loss 0.8802, Train Acc 0.6175, Val Loss 0.9326, Val Acc 0.5835, 
Epoch 10/100: Train Loss 0.9645, Train Acc 0.5122, Val Loss 1.1734, Val Acc 0.3129, 
Epoch 11/100: Train Loss 1.1079, Train Acc 0.3646, Val Loss 1.1199, Val Acc 0.3636, 
Epoch 12/100: Train Loss 1.0684, Train Acc 0.4206, Val Loss 1.0865, Val Ac

In [14]:
# 모델 평가
# - 정답, 모델 예측값을 가지고, classification_report 작성
from sklearn.metrics import classification_report

model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        output = model(X_batch)
        loss = criterion(output, y_batch)
        pred = output.argmax(dim=1)

        all_preds.extend(pred.detach().cpu().numpy())
        all_labels.extend(y_batch.detach().cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=newsgroups.target_names))

                    precision    recall  f1-score   support

     comp.graphics       0.96      0.77      0.85       202
rec.sport.baseball       0.85      0.98      0.91       202
         sci.space       0.84      0.88      0.86       187

          accuracy                           0.88       591
         macro avg       0.88      0.88      0.88       591
      weighted avg       0.89      0.88      0.88       591

